In [ ]:
import pandas as pd
import numpy as np
import sklearn

#importing and cleaning the data
h1 = pd.read_csv("/content/drive/MyDrive/HeartAttack/H1/heart.csv")
o2 = pd.read_csv("/content/drive/MyDrive/HeartAttack/H1/o2Saturation.csv")

h1.drop("0", axis = 1, inplace = True)

h1['age range'] = h1['age range'].apply(lambda x: int(x.split('-')[1]))

from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()

onehotencoder.fit_transform(h1.sex.values.reshape(-1 ,1)).toarray()
onehotencoder.fit_transform(h1.restecg.values.reshape(-1 ,1)).toarray()

h1 = h1.apply(pd.to_numeric, errors = 'coerce' )
#h1.corr().to_csv("/content/drive/MyDrive/HeartAttack/H1/correlation_h1.csv")
#h1[h1.duplicated()]
h1.drop_duplicates(keep='first',inplace=True)
h1.head()

,age,age range,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,29,30,1,1,130,204,0,0,202,0,0.0,2,0,2,1
1,34,40,1,3,118,182,0,0,174,0,0.0,2,0,2,1
2,34,40,0,1,118,210,0,1,192,0,0.7,2,0,2,1
3,35,40,0,0,138,183,0,1,182,0,1.4,2,0,2,1
4,35,40,1,1,122,192,0,1,174,0,0.0,2,0,2,1


In [ ]:
#splitting data into dependent and indepedent
h1_d = h1["output"]
h1_in = h1.drop("output", axis = 1)
print(h1_in.values)

#putting all in range
from sklearn.preprocessing import StandardScaler
standardscaler = StandardScaler()
h1_in = standardscaler.fit_transform(h1_in)

#splitting the data in training and test
from sklearn.model_selection import train_test_split
h1_in_train, h1_in_test, h1_d_train, h1_d_test = train_test_split(h1_in, h1_d, test_size = 0.25, random_state = 2 )


[[29. 30.  1. ...  2.  0.  2.]
 [34. 40.  1. ...  2.  0.  2.]
 [34. 40.  0. ...  2.  0.  2.]
 ...
 [74. 80.  0. ...  2.  1.  2.]
 [76. 80.  0. ...  1.  0.  2.]
 [77. 80.  1. ...  2.  3.  2.]]


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
model_RF = RandomForestClassifier(n_estimators = 64, random_state = 2, max_depth = None, min_samples_split = 2, min_samples_leaf= 1)

model_RF.fit(h1_in_train, h1_d_train)

prediction_RF = model_RF.predict(h1_in_test)

accuracy_RF = accuracy_score( h1_d_test, prediction_RF)
conf_matrix = confusion_matrix(h1_d_test, prediction_RF)

print(f'Accuracy : {accuracy_RF*100}')
print(f'Confusion matrix : \n{conf_matrix}')


Accuracy : 0.868421052631579 #n_estimators = 64

Accuracy : 86.8421052631579
Confusion matrix : 
[[36  5]
 [ 5 30]]


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
base_classifier = DecisionTreeClassifier(max_depth = 1)

Model_AB = AdaBoostClassifier(base_classifier, n_estimators = 71, learning_rate = 0.07, random_state = 2)
Model_AB.fit(h1_in_train, h1_d_train)
prediction_AB = Model_AB.predict(h1_in_test)
accuracy_AB = accuracy_score(h1_d_test, prediction_AB)
print(f'Accuracy : {accuracy_AB}')


Accuracy : 0.8289473684210527


In [ ]:
#SVM
from sklearn.svm import SVC
model_SVM = SVC()
model_SVM.fit(h1_in_train, h1_d_train)
prediction_SVM = model_SVM.predict(h1_in_test)
accuracy_SVM = accuracy_score(h1_d_test, prediction_SVM)

print(f'Accuracy : {accuracy_SVM}')
#Accuracy : 0.8524590163934426 With random_size = 0 and test_size = 0.2

Accuracy : 0.7763157894736842


In [ ]:
from sklearn.linear_model import LogisticRegression
model_LR = LogisticRegression()

model_LR.fit(h1_in_train, h1_d_train)

prediction_LR = model_LR.predict(h1_in_test)

accuracy_LR = accuracy_score(h1_d_test, prediction_LR)
conf_matrix_LR = confusion_matrix(h1_d_test, prediction_LR)
print(f'Accuracy : {accuracy_LR}')
print(f'Confusion matrix: \n{conf_matrix_LR} ')

Accuracy : 0.8289473684210527
Confusion matrix: 
[[34  7]
 [ 6 29]] 


In [ ]:
#gradient boosting
from sklearn.ensemble import GradientBoostingClassifier
model_GB = GradientBoostingClassifier(n_estimators = 225, learning_rate = 0.1)

model_GB.fit(h1_in_train, h1_d_train)
prediction_GB = model_GB.predict(h1_in_test)
accuracy_GB = accuracy_score(h1_d_test, prediction_GB)
print(f'Accuracy : {accuracy_GB}')

Accuracy : 0.75


In [ ]:
#K nearest Neightbour
from sklearn.neighbors import KNeighborsClassifier
model_KNN = KNeighborsClassifier(n_neighbors = 1)~
model_KNN.fit(h1_in_train, h1_d_train)
prediction_KNN = model_KNN.predict(h1_in_test)
accuracy_KNN = accuracy_score(h1_d_test, prediction_KNN)

print(f"Accuracy of KNN is :{accuracy_KNN}")

Accuracy of KNN is :0.7368421052631579



1.   Random Forest
2.   AdaBoosts
1.   SVC
1.   Logistic Regression
2.   Gradiest boosting
6.K Nearest Neighbours

USING GRIDSEARCHCV FOR EVERY ALGO

In [ ]:
#USING GRIDSEARCH CV for RandomForest
from sklearn.model_selection import GridSearchCV
GRF = RandomForestClassifier()

param_grid = {
    'n_estimators' : [50, 64, 70, 100, 150],
    'max_depth' : [None, 10, 20],
    'min_samples_split' : [2, 5, 10],
    'min_samples_leaf' : [1, 2, 4]
}

gridsearch_RF = GridSearchCV(estimator = GRF, param_grid = param_grid, scoring = 'accuracy', cv = 8, n_jobs = -1)
gridsearch_RF.fit(h1_in_train, h1_d_train)
prediction_GRF = gridsearch_RF.predict(h1_in_test)

accuracy_GRF = accuracy_score(h1_d_test, prediction_GRF)
print(f'Accuracy : {accuracy_GRF}')
print()

#print(gridsearch_RF._best_)
print(gridsearch_RF.best_params_)
#{'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 100}
#cv 3 shayad

Accuracy : 0.7894736842105263

{'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}


In [ ]:
#USING GRIDSEARCH CV for AdaBoost
ada = AdaBoostClassifier()
param_grid_ada = { 'n_estimators' : [50, 71, 100, 200], 'learning_rate' : [0.07, 0.1, 0.7, 1]}
gridsearch_ada = GridSearchCV(estimator = ada, param_grid = param_grid_ada,scoring='accuracy', cv=5, n_jobs=-1)
gridsearch_ada.fit(h1_in_train, h1_d_train)
prediction_ada = gridsearch_ada.predict(h1_in_test)
accuracy_ada = accuracy_score(h1_d_test, prediction_ada)
print(f'Accuracy : {accuracy_ada}')

Accuracy : 0.8289473684210527


In [ ]:
#USING GRIDSEARCH CV for SVM

Svc = SVC()

param_grid_svc = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

gridsearch_svc = GridSearchCV(estimator = Svc, param_grid = param_grid_svc ,scoring='accuracy', cv=5, n_jobs=-1)
gridsearch_svc.fit(h1_in_train, h1_d_train)
prediction_svc = gridsearch_svc.predict(h1_in_test)
accuracy_svc = accuracy_score(h1_d_test, prediction_svc)
print(f'Accuracy : {accuracy_svc}')


Accuracy : 0.8157894736842105


In [ ]:
#USING GRIDSEARCH CV for Logisitic Regression

lrr = LogisticRegression()

param_grid_lr = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2']
}

gridsearch_lr = GridSearchCV(estimator = lrr, param_grid = param_grid_lr,scoring='accuracy', cv=5, n_jobs=-1)
gridsearch_lr.fit(h1_in_train, h1_d_train)
prediction_lr = gridsearch_lr.predict(h1_in_test)
accuracy_lr = accuracy_score(h1_d_test, prediction_lr)
print(f'Accuracy : {accuracy_lr}')

Accuracy : 0.8289473684210527


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 54, in _check_solver
    raise ValueError(
ValueError: Solve

In [ ]:
#USING GRIDSEARCH CV for Gradient Boosting

GBM = GradientBoostingClassifier()

param_grid_gb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

gridsearch_gb = GridSearchCV(estimator = GBM, param_grid = param_grid_gb,scoring='accuracy', cv=5, n_jobs=-1)
gridsearch_gb.fit(h1_in_train, h1_d_train)
prediction_gb = gridsearch_gb.predict(h1_in_test)
accuracy_gb = accuracy_score(h1_d_test, prediction_gb)
print(f'Accuracy : {accuracy_gb}')

Accuracy : 0.7368421052631579


In [ ]:
#USING GRIDSEARCH CV for KNearestNeighbor
knn = KNeighborsClassifier()

param_grid_knn = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

gridsearch_knn = GridSearchCV(estimator = knn, param_grid = param_grid_knn,scoring='accuracy', cv=5, n_jobs=-1)
gridsearch_knn.fit(h1_in_train, h1_d_train)
prediction_knn = gridsearch_knn.predict(h1_in_test)
accuracy_knn = accuracy_score(h1_d_test, prediction_knn)
print(f'Accuracy : {accuracy_knn}')

Accuracy : 0.8026315789473685


In [ ]:
from sklearn.ensemble import VotingClassifier

voting_classifier = VotingClassifier(estimators = [('Rforest',model_RF),('Ada',Model_AB),('LRegress',model_LR)],voting = 'soft' )
voting_classifier.fit(h1_in_train, h1_d_train)
prediction_vc = voting_classifier.predict(h1_in_test)
accuracy_vc = accuracy_score(h1_d_test, prediction_vc)

print(f"Accuracy : {accuracy_vc}")

Accuracy : 0.868421052631579
